In [299]:
import bs4 as bs
import urllib.request
import bz2
import json

In [300]:
file = bz2.open('RS_2017-01.bz2')

In [301]:
i = 3
submissions = []
x = ""
for line in file :
    if i > 0 :
        i -= 1
        x = '{'+str(line)[3:-4]+'}'
        y = json.loads(x.replace('\\"','').replace('\\',''), strict=False)
        submissions.append(y)
    else :
        break

In [302]:
submissions[0]

{'link_flair_css_class': None,
 'thumbnail': 'https://b.thumbs.redditmedia.com/ZchXykM_bboD2RUWn8rzxyoyl8keqK4YAS9WLQoEfKg.jpg',
 'created_utc': 1483228800,
 'spoiler': False,
 'locked': False,
 'distinguished': None,
 'hidden': False,
 'stickied': False,
 'quarantine': False,
 'retrieved_on': 1489388027,
 'domain': 'smoothcomp.com',
 'secure_media': None,
 'secure_media_embed': {},
 'title': 'South Dakota event next weekend come compete!',
 'author': 'sonicbh',
 'id': '5lcgj8',
 'num_comments': 0,
 'permalink': '/r/bjj/comments/5lcgj8/south_dakota_event_next_weekend_come_compete/',
 'selftext': '',
 'archived': False,
 'url': 'https://smoothcomp.com/event/128',
 'subreddit': 'bjj',
 'contest_mode': False,
 'subreddit_id': 't5_2qn02',
 'media': None,
 'score': 3,
 'suggested_sort': None,
 'author_flair_css_class': 'black-belt-confirmed',
 'hide_score': False,
 'preview': {'enabled': False,
  'images': [{'source': {'height': 630,
     'url': 'https://i.redditmedia.com/RDYuH2-9btvlKhGaLQ

In [308]:
for submission in submissions :
    filename = submission['id'] + '.xml'
    f = open(filename,"w+")
    
    f.write("<?xml version='1.0' encoding='utf-8'?>")
    f.write("<KnolML>")
    f.write('	<KnowledgeData Type="Submision" Id= "1">')
    
    f.write('		<Title>'+submission['title']+'</Title>')
    f.write('		<Instance Id= "1" InstanceType= "Description">')
    
    f.write("			<TimeStamp>")
    f.write(" 				<CreationUTC>"+str(submission['created_utc'])+"</CreationUTC> ")
    f.write("			</TimeStamp>")
    
    f.write("			<Contributors>")
    f.write(" 				<OwnerUserId>"+submission['author']+"</OwnerUserId> ")
    f.write("			</Contributors>")
    
    f.write("			<Body>")
    f.write(" 				<Text>")
    f.write("					"+submission['selftext'])
    f.write("				</Text> ")
    f.write("			</Body>")
    
    f.write("			<Credit>")
    f.write(" 				<Score>"+str(submission['score'])+"</Score> ")
    f.write("			</Credit>")
    
    f.write('		</Instance>')

    submission_id = submission['id']
    url = 'https://api.pushshift.io/reddit/submission/comment_ids/' + submission_id
    sauce = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(sauce, 'lxml')
    
    comment_ids = json.loads(soup.p.string)['data']
    
    for comment_id in comment_ids :
        url = 'http://api.pushshift.io/reddit/search/comment/?ids=' + comment_id
        sauce = urllib.request.urlopen(url).read()
        soup = bs.BeautifulSoup(sauce, 'lxml')
    
        comment_description = json.loads(soup.p.string)['data']        
        for x in comment_description :
            f.write('		<Instance Id= "2" OriginalId= "4" InstanceType= "Answer">')
    
    f.write('	</KnowledgeData>')
    f.write('</KnolML>')

[{'author': 'ColonelGiraffi', 'author_flair_css_class': None, 'author_flair_text': None, 'body': 'Super fucking bleak for a Star Wars film. Liked that it went right up to the start of ep 4 though. ', 'created_utc': 1483228926, 'id': 'dbumqh4', 'link_id': 't3_5lcgj9', 'parent_id': 't3_5lcgj9', 'stickied': False, 'subreddit': 'AskReddit', 'subreddit_id': 't5_2qh1i'}]
ColonelGiraffi
[{'author': 'Tomcat1108', 'author_flair_css_class': None, 'author_flair_text': None, 'body': "Can't wait for the sequel.  ", 'created_utc': 1483228931, 'id': 'dbumqla', 'link_id': 't3_5lcgj9', 'parent_id': 't3_5lcgj9', 'stickied': False, 'subreddit': 'AskReddit', 'subreddit_id': 't5_2qh1i'}]
Tomcat1108
[{'author': 'pm_me_anythingg_sfw', 'author_flair_css_class': None, 'author_flair_text': None, 'body': 'So mindblowing', 'created_utc': 1483228932, 'id': 'dbumqm3', 'link_id': 't3_5lcgj9', 'parent_id': 't3_5lcgj9', 'stickied': False, 'subreddit': 'AskReddit', 'subreddit_id': 't5_2qh1i'}]
pm_me_anythingg_sfw
[{'au

In [ ]:
sauce = urllib.request.urlopen('http://api.pushshift.io/reddit/search/comment/?ids=cnas901').read()
soup = bs.BeautifulSoup(sauce, 'lxml')

In [ ]:
print(soup.p.string)